In [60]:
import numpy as np
import pandas as pd
import os
import re 

In [61]:
posts_full = pd.read_csv('posts_full.csv')

In [62]:
comments_full = pd.read_csv('comments_full.csv')

In [63]:
posts = posts_full
comments = comments_full

display(posts.head())
display(comments.head())

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ttdab1,2s5ti,meirl,False,1648771149,https://old.reddit.com/r/meirl/comments/ttdab1...,i.imgur.com,https://i.imgur.com/ucZiw34.jpg,NaN,meirl,93
1,post,ttd0r4,2s5ti,meirl,False,1648770296,https://old.reddit.com/r/meirl/comments/ttd0r4...,i.redd.it,https://i.redd.it/x8sehlq23tq81.jpg,NaN,Me_irl,1
2,post,ttd04u,2s5ti,meirl,False,1648770242,https://old.reddit.com/r/meirl/comments/ttd04u...,i.redd.it,https://i.redd.it/p86ehgxw2tq81.jpg,NaN,Meirl,605
3,post,ttc7og,2s5ti,meirl,False,1648767776,https://old.reddit.com/r/meirl/comments/ttc7og...,imgur.com,https://imgur.com/KIU84VZ.jpeg,NaN,meirl,67
4,post,ttc58g,2s5ti,meirl,False,1648767600,https://old.reddit.com/r/meirl/comments/ttc58g...,i.redd.it,https://i.redd.it/to2ot762vsq81.jpg,NaN,me irl,32


,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,i2x2j0g,2s5ti,meirl,False,1648771186,https://old.reddit.com/r/meirl/comments/tswh3j...,Yes,NaN,1
1,comment,i2x2hqk,2s5ti,meirl,False,1648771171,https://old.reddit.com/r/meirl/comments/tt9v20...,It’s worse if you’re a man,-0.4767,5
2,comment,i2x2dv0,2s5ti,meirl,False,1648771121,https://old.reddit.com/r/meirl/comments/tt5aas...,They took our jerbs!!!!,0.0000,3
3,comment,i2x2d15,2s5ti,meirl,False,1648771110,https://old.reddit.com/r/meirl/comments/ttbnpo...,It certainly can help you from a mental standp...,0.7650,3
4,comment,i2x28i1,2s5ti,meirl,False,1648771053,https://old.reddit.com/r/meirl/comments/tswh3j...,Bottom picture,0.0000,1


In [64]:
display(posts.columns)
display(posts.shape)
display(comments.columns)
display(comments.shape)

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'domain', 'url', 'selftext', 'title',
       'score'],
      dtype='object')

(3362747, 12)

Index(['type', 'id', 'subreddit.id', 'subreddit.name', 'subreddit.nsfw',
       'created_utc', 'permalink', 'body', 'sentiment', 'score'],
      dtype='object')

(12053027, 10)

In [65]:
comments['id_post'] = comments.permalink.str.extract(r'https://old.reddit.com/r/meirl/comments/([^/]+)/meirl/.*', expand=False)
comments['created_hour'] = pd.to_datetime(comments['created_utc'], unit='s', origin='unix').dt.hour

In [66]:
posts_mean_score = posts['score'].mean()
display(posts_mean_score)

240.03112485120053

In [67]:
comments[comments['sentiment'] < -0.9].groupby('created_hour').size().reset_index(name='qty').sort_values(by='qty', ascending=False).head(10)

,created_hour,qty
15,15,2134
16,16,2091
18,18,2044
17,17,2040
20,20,2004
19,19,1996
22,22,1951
14,14,1937
21,21,1869
23,23,1820


In [68]:
posts_to_join = posts[['id', 'selftext', 'url', 'permalink', 'score']]
comments_to_join = comments[['id_post', 'body', 'sentiment', 'created_hour']]
comments_to_join = comments_to_join[(comments['body'].notnull()) & (comments['body'] != 'NaN')]
comments_with_posts = posts_to_join.set_index('id').join(comments_to_join.set_index('id_post'), how='inner')

In [69]:
display(comments_with_posts)

,selftext,url,permalink,score,body,sentiment,created_hour
2dlhff,NaN,http://imgur.com/0NHshtG,https://old.reddit.com/r/meirl/comments/2dlhff...,19,frig off,NaN,13
2nfgu1,NaN,http://i.imgur.com/Ldg6D2y.jpg,https://old.reddit.com/r/meirl/comments/2nfgu1...,7,"I soar on an updraft, and am ever twirling, tw...",0.0000,21
2nfgu1,NaN,http://i.imgur.com/Ldg6D2y.jpg,https://old.reddit.com/r/meirl/comments/2nfgu1...,7,this speaks to me like a plastic bag blowing i...,0.3612,21
2rispp,NaN,http://i.imgur.com/qgdbTpB.gif,https://old.reddit.com/r/meirl/comments/2rispp...,31,Sauce?,NaN,9
2rispp,NaN,http://i.imgur.com/qgdbTpB.gif,https://old.reddit.com/r/meirl/comments/2rispp...,31,Don't we all,0.0000,17
...,...,...,...,...,...,...,...
ttbv93,NaN,https://i.redd.it/bxfvfw1issq81.jpg,https://old.reddit.com/r/meirl/comments/ttbv93...,144,This is true however it's never safe to assume...,0.5505,23
ttbv93,NaN,https://i.redd.it/bxfvfw1issq81.jpg,https://old.reddit.com/r/meirl/comments/ttbv93...,144,"Yeah, that chair will fit perfect is the corne...",0.8126,22
ttbv93,NaN,https://i.redd.it/bxfvfw1issq81.jpg,https://old.reddit.com/r/meirl/comments/ttbv93...,144,Especially since we know you live alone. No re...,-0.4939,22
ttbv93,NaN,https://i.redd.it/bxfvfw1issq81.jpg,https://old.reddit.com/r/meirl/comments/ttbv93...,144,Exactly you don't go putting yourself in a fla...,0.3612,22


In [21]:
student_comments = comments_with_posts[(comments_with_posts['body'].str.contains('university', flags=re.IGNORECASE, regex=True)) |
                                     (comments_with_posts['body'].str.contains('college', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('student', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('teacher', flags=re.IGNORECASE, regex=True)) | 
                                     (comments_with_posts['body'].str.contains('professor', flags=re.IGNORECASE, regex=True))]
display(student_comments)

,selftext,url,permalink,score,body,sentiment,created_hour
3zw04q,NaN,https://i.imgur.com/wgfQy9j.jpg,https://old.reddit.com/r/meirl/comments/3zw04q...,141,"Holy mother of buzzwords, this is worse than c...",-0.4767,0
40m9es,[deleted],NaN,https://old.reddit.com/r/meirl/comments/40m9es...,4109,"Can confirm, born in 97 and am a freshman in c...",0.0000,22
40m9es,[deleted],NaN,https://old.reddit.com/r/meirl/comments/40m9es...,4109,Someone born yesterday won't be entering colle...,0.0000,22
40m9es,[deleted],NaN,https://old.reddit.com/r/meirl/comments/40m9es...,4109,"Some kids born in '97 are in college now, so i...",0.0000,22
42fqkn,NaN,http://i.imgur.com/cx7Ak8a.jpg,https://old.reddit.com/r/meirl/comments/42fqkn...,801,Is that Brian Murphy from College humor?,0.2732,23
...,...,...,...,...,...,...,...
tsln86,NaN,https://i.redd.it/edpns7wtplq81.jpg,https://old.reddit.com/r/meirl/comments/tsln86...,3371,"Yes because stay at home being a nurse, elderl...",-0.2163,8
tsn173,[deleted],NaN,https://old.reddit.com/r/meirl/comments/tsn173...,129,So true teachers hate on us rapers i dont get ...,-0.5520,7
tswh3j,NaN,https://i.redd.it/lxhymxn11pq81.jpg,https://old.reddit.com/r/meirl/comments/tswh3j...,143559,Once had the best train ride sitting next to a...,0.6124,18
tswh3j,NaN,https://i.redd.it/lxhymxn11pq81.jpg,https://old.reddit.com/r/meirl/comments/tswh3j...,143559,Worst part is I work in a university.. I have ...,-0.7530,17


In [81]:
student_comments_best_urls = student_comments[student_comments['score'] > posts_mean_score]['url']
display(student_comments_best_urls.shape)

(9128,)

In [92]:
df = pd.DataFrame(data=student_comments_best_urls).drop_duplicates()
df.shape

(1816, 1)

In [93]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

                                                      url
40m9es                                                NaN
42fqkn                     http://i.imgur.com/cx7Ak8a.jpg
438nro                     http://i.imgur.com/jGTlwDD.jpg
44llao                     http://i.imgur.com/YdvXrdD.jpg
44qzad                    https://i.imgur.com/t7FtM5o.gif
47c93z                      https://imgur.com/BZmJWBp.jpg
492q8d  https://56.media.tumblr.com/9a3728a8a4c6016ac4...
49ln7u                     http://i.imgur.com/yvR5zT1.png
4c05kk                    http://i.imgur.com/p24ioZI.gifv
4c66nb                     http://i.imgur.com/xMu6I9n.png
4cgjyk                     http://i.imgur.com/nDku8hN.jpg
4ciccp                          https://imgur.com/a/SwkBk
4ck53p                    https://i.imgur.com/seOEGDe.jpg
4fhgbi                     http://i.imgur.com/QDdInr6.jpg
4fsgnd                     http://i.imgur.com/4q3YqD8.jpg
4gco7a                    http://i.imgur.com/paDgHmV.gifv
4h5nzl        

In [94]:
memes_with_sentiment = comments_with_posts[['url', 'sentiment']].groupby('url').mean().reset_index().dropna().sort_values('sentiment', ascending=False)
display(memes_with_sentiment)

,url,sentiment
3644,http://i.imgur.com/qx2FNm4.jpg,0.9992
45315,https://i.redd.it/lc201s9bt9s11.jpg,0.9943
41735,https://i.redd.it/iqyhy207rp851.jpg,0.9905
13338,https://i.imgur.com/gXTndtD.png,0.9905
31260,https://i.redd.it/bb30b8an76361.jpg,0.9899
...,...,...
10803,https://i.imgur.com/KmKCfJf.jpg,-0.9796
33324,https://i.redd.it/csgn1tc90oc31.jpg,-0.9805
40358,https://i.redd.it/hrnlfifepwe41.png,-0.9828
48707,https://i.redd.it/ntc1ul3tgyn81.jpg,-0.9855
